In [ ]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display

In [ ]:
import struct

class WavFileHelper():
    
    def read_file_properties(self, filename):

        wave_file = open(filename,"rb")
        
        riff = wave_file.read(12)
        fmt = wave_file.read(36)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I",sample_rate_string)[0]
        
        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H",bit_depth_string)[0]

        return (num_channels, sample_rate, bit_depth)

wavfilehelper = WavFileHelper()

In [1]:
def extract_features(file_name):
   
    try:
        audio_y, sample_rate = librosa.load(file_name, res_type='kaiser_best') 
        D = (librosa.stft(audio_y, center=False, n_fft=1024))
        
        librosa.display.specshow(librosa.amplitude_to_db(D,
                                                 ref=np.max),
                         y_axis='log', x_axis='time')
        plt.title('Power spectrogram')
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        plt.show()
        
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return (D)

In [ ]:
metadata = pd.read_csv('./UrbanSound8k/metadata/UrbanSound8K.csv')
features = []
i = 0

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    i = i +1
    file_name = os.path.join(os.path.abspath('UrbanSound8k/audio/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class"]
    print(f'{i}: Processing {file_name}')
    data = extract_features(file_name)
    
    features.append(data)

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(features), ' files')